In [1]:
#| default_exp 05_map-amazon-meta-from-dump

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [4]:
#| export
from timeit import default_timer as timer
import json, gzip, argparse, os, pickle
from functools import partial
from tqdm.auto import tqdm

In [6]:
#| export
from sugar import *

In [7]:
#| export
def default_condition(o): return True

def read_jsongz(fname, condition=default_condition):
    with gzip.open(fname, 'rt', encoding='utf-8') as file:
        return [json.loads(o) for o in file if condition(json.loads(o))]

def read_jsonl(fname, condition=default_condition):
    with open(fname, 'r') as file:
        return [json.loads(o) for o in file if condition(json.loads(o))]

def _read_file(fname, condition=default_condition):
    if fname.endswith('.jsonl'):
        return read_jsonl(fname, condition)
    elif fname.endswith('.json.gz'):
        return read_jsongz(fname, condition)
    else:
        raise ValueError(f'Invalid file: {fname}')

def _get_relevant_items(raw_dir, condition=default_condition):
    all_items = []
    for file in tqdm([f'{raw_dir}/{fn}' for fn in os.listdir(raw_dir)]):
        all_items.extend(_read_file(file, condition))
    return all_items

def condition_a23(o, ids, id_field):
    return o[id_field] in ids

def get_condition(data_dir, dtype, id_field, encoding='utf-8'):
    all_ids = get_all_ids(f'{data_dir}/raw_data', encoding)
    if dtype == 'a23': return partial(condition_a23, ids=all_ids, id_field=id_field)
    elif dtype is None: return default_condition
    else: raise ValueError(f'Invalid condition: {dtype}')

def file_exists(fname):
    try:
        with open(fname, 'rb') as file:
            all_items = pickle.load(file)
        return True
    except:
        return False

In [8]:
#| export
def get_relevant_items(data_dir, raw_dir, id_field, condition_type=None, encoding='utf-8'):
    fname = f'{data_dir}/all_products.pkl' if condition_type is None else f'{data_dir}/all_products_{condition_type}.pkl'

    if file_exists(fname):
        with open(fname, 'rb') as file:
            all_items = pickle.load(file)
    else:
        condition = get_condition(data_dir, condition_type, id_field, encoding)
        all_items = _get_relevant_items(raw_dir, condition)
        with open(fname, 'wb') as file:
            pickle.dump(all_items, file)

    return all_items

def title_proc(o): return [o['title']]
def description_proc(o): return o['description']
def details_proc(o): return [o['details'].__str__()]

def image_proc(o):
    images = []
    for image in o['images']:
        found = False
        for dtype in ['large', 'hi_res', 'thumb']:
            if dtype in image and image[dtype] is not None:
                images.append(image[dtype])
                found = True
                break
        if not found: raise ValueError('Image not found')
    return images

In [9]:
#| export
def get_proc_type(dtype):
    if dtype == 'description': return description_proc
    elif dtype == 'image': return image_proc
    elif dtype == 'details': return details_proc
    else: raise ValueError(f'Invalid function type: {dtype}')

def extract_meta_info(all_items, meta_type, id_field):
    proc_func = get_proc_type(meta_type)

    meta_info = {}
    for o in all_items:
        meta_info[o[id_field]] = proc_func(o)
    return meta_info

In [ ]:
#| export
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--raw_dir', type=str, required=True)
    parser.add_argument('--data_dir', type=str, required=True)
    parser.add_argument('--id_field', type=str, required=True)
    parser.add_argument('--condition_type', type=str, default=None)
    parser.add_argument('--encoding', type=str, default='utf-8')
    parser.add_argument('--meta_type', type=str, required=True)
    return parser.parse_args()

if __name__ == '__main__':
    start_time = timer()

    args = parse_args()

    all_items = get_relevant_items(args.data_dir, args.raw_dir, args.id_field,
            args.condition_type, args.encoding)

    meta_info = extract_meta_info(all_items, args.meta_type, args.id_field)
    matrix, ids, vocab = get_matrix_from_mapping(meta_info)

    end_time = timer()
    print(f'Time elapsed: {end_time-start_time:.2f} seconds.')
    